Digital Skola, Batch 11

Spark HomeWork

M. Abdurrahman Shidiq

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
#Create a spark session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 14:29:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/20 14:29:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Extract
 Extract process: read data tsb ke dalam csv dan dijadikan dataframe

In [3]:
df = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("fhv_tripdata_2019-01.csv.gz")
print("Here is our inferred schema:")
df.printSchema()

Here is our inferred schema:
root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [4]:
df.show() 

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00001|2019-01-01 00:30:00|2019-01-01 02:51:55|        null|        null|   null|                B00001|
|              B00001|2019-01-01 00:45:00|2019-01-01 00:54:49|        null|        null|   null|                B00001|
|              B00001|2019-01-01 00:15:00|2019-01-01 00:54:52|        null|        null|   null|                B00001|
|              B00008|2019-01-01 00:19:00|2019-01-01 00:39:00|        null|        null|   null|                B00008|
|              B00008|2019-01-01 00:27:00|2019-01-01 00:37:00|        null|        null|   null|                B00008|
|              B00008|2019-01-01 00:48:0

In [5]:
df.count()

23143222

# Transform
Transforms process: filter data dimana kolom `PUlocationID` dan `DOlocationID` **tidak boleh kosong** dan `pickup_datetime` mulai dari **1 Jan 2019 sampai 10 Jan 2019**

In [6]:
df.createOrReplaceTempView("tripData")

In [7]:
query = """
SELECT
    *
FROM
    tripData
WHERE
    PUlocationID IS NOT NULL
    AND DOlocationID IS NOT NULL
    AND DATE(pickup_datetime) BETWEEN '2019-01-01' AND '2019-01-10'
ORDER BY 4,5,2
"""

sqlDF = spark.sql(query)
sqlDF.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02832|2019-01-02 09:00:00|2019-01-02 09:30:00|           0|           0|   null|                B02832|
|              B02832|2019-01-02 10:00:00|2019-01-02 10:30:00|           0|           0|   null|                B02832|
|              B02832|2019-01-02 10:00:00|2019-01-02 10:30:00|           0|           0|   null|                B02832|
|              B02832|2019-01-02 10:30:00|2019-01-02 11:00:00|           0|           0|   null|                B02832|
|              B02832|2019-01-02 11:00:00|2019-01-02 11:30:00|           0|           0|   null|                B02832|
|              B02832|2019-01-02 11:30:0

In [8]:
sqlDF.count()

6006104

# Load
Load Process : write data setelah proses transform ke parquet dan json file

In [10]:
# PARQUET
sqlDF.write.parquet("spark_write_parquet.parquet")

# JSON
sqlDF.write.parquet("spark_write_json.json")

23/02/20 14:32:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
